In [129]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
from pymongo import MongoClient
import pprint
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

In [161]:
#Connect to hosted MongoDB instance
client = MongoClient()
#Access already existing database
db = client[
    'capstone2_updated']
#Access already existing collection in database
diamond_info = db['diamond_info']

In [261]:
#Create web browser (do not need to have driver install - Safari driver for mac 
#is preloaded on the OS. Just need to enable: /usr/bin/safaridriver --enable)
driver = webdriver.Safari()
#Go to desired URL
driver.get('https://www.bluenile.com/diamond-search')
sleep(3)

Lazy loader only gives 1000 rows of data and iterating through pages only gives up to 42 pages of data. Work-around - change price, grab 1000 rows, update price floor to previous pulled max and pull again. Repeat.

In [262]:
#expanding the menu for all filters
expanded_filters = driver.find_element_by_xpath('//*[@id="react-app"]/div/div/section/div/div[2]/div[3]/button[2]')
driver.execute_script("arguments[0].click()",expanded_filters)
sleep(1)

#toggling on all additional filters - unclicks round: removes it
filters = ['Round-filter-button-med-lrg','Princess-filter-button-med-lrg','Emerald-filter-button-med-lrg',
             'Asscher-filter-button-med-lrg','Cushion-filter-button-med-lrg','Marquise-filter-button-med-lrg',
             'Radiant-filter-button-med-lrg','Oval-filter-button-med-lrg','Pear-filter-button-med-lrg',
             'Heart-filter-button-med-lrg','polish-toggle-button','symmetry-toggle-button',
           'fluorescence-toggle-button','depth %-toggle-button','table %-toggle-button','l/w ratio-toggle-button']
for element_id in filters:
    feature = driver.find_element_by_id(element_id)
    driver.execute_script("arguments[0].click()",feature)
    sleep(1)
cutlet=driver.find_element_by_xpath('//*[@id="culet-filter-container-accordion"]/button')
driver.execute_script("arguments[0].click()",cutlet)

## Pull first two pages for example to see if it works (322 is the max price from first place to use as the floor price for next page)

In [263]:
driver.execute_script("window.scrollTo(0, window.scrollY + 64400)") # scroll down the page - lazy loader
sleep(12) #need to wait for page to load
# for the initial page with default filters 
page_source = driver.page_source
diamond_info.insert_one({'pull':1,'html':page_source})

#initiate first max price
diamonds = diamond_info.find_one({'pull':1})['html']
soup = BeautifulSoup(diamonds,'html.parser')
if len(soup.find_all('div','row-cell price')[999].text.split('Now: ')) == 2:
    x =  soup.find_all('div','row-cell price')[999].text.split('Now: ')[0][5:]
else: 
    x = soup.find_all('div','row-cell price')[999].text
max_price = int(x[1:]) 

#initialize pull to keep count
pull = 1

#set previous max price to the next pulls floor price to update
while max_price < 326:
    #scroll up the page to update floor price
    driver.execute_script("window.scrollTo(0, window.scrollY - 65000)")
    #updating price
    inputElement = driver.find_element_by_xpath("/html/body/div[1]/main/div/div/div/section/div/div[2]/div[3]/div[7]/div[2]/div/fieldset/div/div[1]/input")
    inputElement.send_keys(max_price)
    inputElement.send_keys(Keys.ENTER)
    #scrolling down page for updated data 
    driver.execute_script("window.scrollTo(0, window.scrollY + 65000)")
    sleep(12)
    #insert into mongodb
    page_source = driver.page_source
    pull+=1
    diamond_info.insert_one({'pull':pull,'html':page_source})
    #updating max price for next iteration 
    diamonds = diamond_info.find_one({'pull':pull})['html']
    soup = BeautifulSoup(diamonds,'html.parser')
    if len(soup.find_all('div','row-cell price')[-1].text.split('Now: ')) == 2:
            price = soup.find_all('div','row-cell price')[-1].text.split('Now: ')[0][6:].replace(',','')
            max_price =  int(price)
        else: 
            price = soup.find_all('div','row-cell price')[-1].text[1:].replace(',','')
            max_price = int(price)

In [329]:
def scraper(total_max_price):
    
    client = MongoClient()
    db = client['capstone2_updated']
    diamond_info = db['diamond_info']
    
    
    driver = webdriver.Safari()
    driver.get('https://www.bluenile.com/diamond-search')
    sleep(3)
    
    expanded_filters = driver.find_element_by_xpath('//*[@id="react-app"]/div/div/section/div/div[2]/div[3]/button[2]')
    driver.execute_script("arguments[0].click()",expanded_filters)
    sleep(1)

    filters = ['Round-filter-button-med-lrg','Princess-filter-button-med-lrg','Emerald-filter-button-med-lrg',
                 'Asscher-filter-button-med-lrg','Cushion-filter-button-med-lrg','Marquise-filter-button-med-lrg',
                 'Radiant-filter-button-med-lrg','Oval-filter-button-med-lrg','Pear-filter-button-med-lrg',
                 'Heart-filter-button-med-lrg','polish-toggle-button','symmetry-toggle-button',
               'fluorescence-toggle-button','depth %-toggle-button','table %-toggle-button','l/w ratio-toggle-button']
    for element_id in filters:
        feature = driver.find_element_by_id(element_id)
        driver.execute_script("arguments[0].click()",feature)
        sleep(1)
    cutlet=driver.find_element_by_xpath('//*[@id="culet-filter-container-accordion"]/button')
    driver.execute_script("arguments[0].click()",cutlet)
    
    
    driver.execute_script("window.scrollTo(0, window.scrollY + 64400)")
    sleep(12) 
    page_source = driver.page_source
    diamond_info.insert_one({'pull':1,'html':page_source})


    diamonds = diamond_info.find_one({'pull':1})['html']
    soup = BeautifulSoup(diamonds,'html.parser')
    if len(soup.find_all('div','row-cell price')[-1].text.split('Now: ')) == 2:
        x =  soup.find_all('div','row-cell price')[-1].text.split('Now: ')[0][5:]
    else: 
        x = soup.find_all('div','row-cell price')[-1].text
    max_price = int(x[1:]) 


    pull = 1
    while max_price < total_max_price:
        driver.execute_script("window.scrollTo(0, window.scrollY - 65000)")
        inputElement = driver.find_element_by_xpath("/html/body/div[1]/main/div/div/div/section/div/div[2]/div[3]/div[7]/div[2]/div/fieldset/div/div[1]/input")
        inputElement.send_keys(max_price)
        inputElement.send_keys(Keys.ENTER)
        driver.execute_script("window.scrollTo(0, window.scrollY + 65000)")
        sleep(12)
        page_source = driver.page_source
        pull+=1
        diamond_info.insert_one({'pull':pull,'html':page_source})
        diamonds = diamond_info.find_one({'pull':pull})['html']
        soup = BeautifulSoup(diamonds,'html.parser')
        if len(soup.find_all('div','row-cell price')[-1].text.split('Now: ')) == 2:
            price = soup.find_all('div','row-cell price')[-1].text.split('Now: ')[0][6:].replace(',','')
            max_price =  int(price)
        else: 
            price = soup.find_all('div','row-cell price')[-1].text[1:].replace(',','')
            max_price = int(price)
    
    return "Done!"

In [330]:
scraper(2702260) #max price taken from website

'Done!'